####Cristiano Moreira Martins

In [1]:
import pandas as pd
import plotly.express as px

from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics.pairwise import linear_kernel

c:\Users\crist\.conda\envs\trading\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\crist\.conda\envs\trading\lib\site-packages\numpy\.libs\libopenblas.4SP5SUA7CBGXUEOC35YP2ASOICYYEQZZ.gfortran-win_amd64.dll
c:\Users\crist\.conda\envs\trading\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
data = pd.read_parquet('./funds_metrics.parquet')

data = data[data['vol'] <=1]
data['class'] =  pd.cut(data['vol'], bins=[0, 0.01, 0.02, 0.08, 0.10, data['vol'].max()], labels=[1,2,3,4,5])

data.reset_index(inplace=True)
data.drop(columns=['index'], inplace=True)

print(len(data))
data.head(6)

8208


,cnpj,return,avg_return,vol,max_dd,class
0,00068305000135,0.389372,0.068037,0.003425,-0.003986,1
1,00071477000168,0.329269,0.058625,0.002791,-0.000071,1
2,00073041000108,0.412743,0.071609,0.002711,-0.000731,1
3,00083181000167,0.776607,0.121910,0.209643,-0.419639,5
4,00089915000115,0.392057,0.068450,0.009317,-0.003807,1
5,00102322000141,-0.096708,-0.020152,0.347817,-0.758515,5


In [3]:
px.scatter(data, x='return', y='vol', color='class', width=1000)#, color_discrete_sequence=px.colors.cyclical.Edge)

In [4]:
px.bar(data, x='class', width=1000)

In [5]:
columns = ['return', 'avg_return', 'vol', 'max_dd', 'class']
cosine_similarities = linear_kernel(data[columns],data[columns])
indices = pd.Series(data.index, index=data['cnpj']).drop_duplicates()

#busca uma lista de fundos utilizando a similaridade do cosseno
def getFundsList(cnpj, size):
    idx = indices[cnpj]

    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    top_similar = sim_scores[1:size + 1]
    cnpj_indices = [i[0] for i in top_similar]
    cnpj_indices = [data.iloc[i]['cnpj'] for i in cnpj_indices]

    return cnpj_indices

#busca fundos do mesmo risco do usuario
def getFundsNewUser(user_risk, size):
    return data[data['class'] == user_risk].sample(n=size)

#### 1-lista de recomendações para usuário antigo

In [6]:
#risco inicial em 1
risks = [0, 0.01, 0.02, 0.08, 0.10]
users = pd.DataFrame()

id = 1
user_risk = 1
funds_list = data[(data['vol'] >= risks[user_risk-1]) & (data['vol'] < risks[user_risk])].sample(n=5)
portfolio_risk = funds_list['vol'].mean()

user = {
    'userid': id,
    'user_risk': user_risk,
    'portfolio_risk': portfolio_risk,
    'funds_list': funds_list
}

user

{'userid': 1,
 'user_risk': 1,
 'portfolio_risk': 0.006701628347434764,
 'funds_list':                 cnpj    return  avg_return       vol    max_dd class
 2555  11391262000140  0.481189    0.081805  0.006660 -0.014652     1
 5531  23738226000190  0.519416    0.087337  0.009773 -0.063362     1
 4406  18716322000105  0.394553    0.068833  0.004043 -0.010543     1
 6717  28152799000150  0.453175    0.077679  0.005521 -0.020413     1
 280   02887434000180  0.470124    0.080183  0.007511 -0.020176     1}

In [7]:
users = pd.concat([users, pd.DataFrame([user])])

In [8]:
user['funds_list']

,cnpj,return,avg_return,vol,max_dd,class
2555,11391262000140,0.481189,0.081805,0.006660,-0.014652,1
5531,23738226000190,0.519416,0.087337,0.009773,-0.063362,1
4406,18716322000105,0.394553,0.068833,0.004043,-0.010543,1
6717,28152799000150,0.453175,0.077679,0.005521,-0.020413,1
280,02887434000180,0.470124,0.080183,0.007511,-0.020176,1


In [9]:
#recomenda uma lista para usuario antigo
cnpj_user = user['funds_list'].sample(n=1)['cnpj'].values[0]
rec_user = getFundsList(cnpj_user, 5)
data[data['cnpj'].isin(rec_user)]

,cnpj,return,avg_return,vol,max_dd,class
1714,08843959000109,12.501519,0.683667,0.322012,-0.267813,5
4629,19599474000120,10.863123,0.640629,0.239518,-0.132979,5
5342,22975153000197,26.305992,0.938562,0.342722,-0.200510,5
5374,23129095000143,12.663850,0.687699,0.586140,-0.725603,5
7888,31933729000144,11.256633,0.651380,0.648862,-0.261140,5


#### 2 -lista de recomendações para usuário novo

In [10]:
#recomendacao para novo usuario
id = 2
user_risk = 3
funds_list = getFundsNewUser(user_risk, 5)
portfolio_risk = funds_list['vol'].mean()

user = {
    'userid': id,
    'user_risk': user_risk,
    'portfolio_risk': portfolio_risk,
    'funds_list': funds_list
}

user

{'userid': 2,
 'user_risk': 3,
 'portfolio_risk': 0.05675119048302578,
 'funds_list':                 cnpj    return  avg_return       vol    max_dd class
 3866  17092251000146  0.297586    0.053526  0.062106 -0.112802     3
 3107  13397735000142  0.062828    0.012271  0.077433 -0.159460     3
 3284  13966590000153  0.473343    0.080656  0.032731 -0.053344     3
 1469  07967589000140  0.457244    0.078282  0.060330 -0.132094     3
 1575  08532563000132  0.406173    0.070610  0.051156 -0.118504     3}

In [11]:
#recomendações
user['funds_list']

,cnpj,return,avg_return,vol,max_dd,class
3866,17092251000146,0.297586,0.053526,0.062106,-0.112802,3
3107,13397735000142,0.062828,0.012271,0.077433,-0.159460,3
3284,13966590000153,0.473343,0.080656,0.032731,-0.053344,3
1469,07967589000140,0.457244,0.078282,0.060330,-0.132094,3
1575,08532563000132,0.406173,0.070610,0.051156,-0.118504,3


In [12]:
users = pd.concat([users, pd.DataFrame([user])])

#### 3-recomendação para usuário antigo

In [13]:
#recomenda item novo para usuario antigo
id = 3
user_risk = 4
funds_list = data[(data['vol'] >= risks[user_risk-1]) & (data['vol'] < risks[user_risk])].sample(n=5)
portfolio_risk = funds_list['vol'].mean()

cnpj_user = user['funds_list'].sample(n=1)['cnpj'].values[0]
rec_user = getFundsList(cnpj_user, 1)
data[data['cnpj'].isin(rec_user)]

,cnpj,return,avg_return,vol,max_dd,class
5342,22975153000197,26.305992,0.938562,0.342722,-0.20051,5
